In [34]:
import geopandas as gpd
import os
import pandas as pd
import folium
import dotenv
from folium.plugins import MarkerCluster, Search
import numpy as np
import requests
from pyproj import Proj
from pyproj import transform

In [35]:
WGS84 = { 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84', }
TM128 = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'128E', 'ellps':'bessel',
   'x_0':'400000', 'y_0':'600000', 'k':'0.9999',
   'towgs84':'-146.43,507.89,681.46'}

# Get csv files

In [36]:
def csvToDataFrame(root_path:str,crs:str)->pd.DataFrame:
    gdf = None
    for (dir,_,file) in os.walk(root_path):
        print(file)
        for f in file:
            if gdf is None:
                gdf = pd.read_csv(dir+f)
            else:
                gdf= pd.concat([gdf,pd.read_csv(dir+f)],ignore_index=True)

    gdf = gpd.GeoDataFrame(gdf,geometry=gpd.points_from_xy(gdf.lng,gdf.lat),crs=crs)
    return gdf

In [37]:
def tm128_to_wgs84(x, y):
   return transform( Proj(**TM128), Proj(**WGS84), x, y )

In [38]:
gdf_by_sgis = csvToDataFrame("./data/geocodeBySGIS/","epsg:4326")
gdf_by_sgis.head()

['traditional_w_coordinate_27500-27600.csv', 'traditional_w_coordinate_15100-15200.csv', 'traditional_w_coordinate_13900-14000.csv', 'traditional_w_coordinate_1700-1800.csv', 'traditional_w_coordinate_14500-14600.csv', 'traditional_w_coordinate_18300-18400.csv', 'traditional_w_coordinate_1100-1200.csv', 'traditional_w_coordinate_26100-26200.csv', 'traditional_w_coordinate_7600-7700.csv', 'traditional_w_coordinate_5000-5100.csv', 'traditional_w_coordinate_3900-4000.csv', 'traditional_w_coordinate_8000-8100.csv', 'traditional_w_coordinate_1800-1900.csv', 'traditional_w_coordinate_10100-10200.csv', 'traditional_w_coordinate_19400-19500.csv', 'traditional_w_coordinate_22500-22600.csv', 'traditional_w_coordinate_23100-23200.csv', 'traditional_w_coordinate_7700-7800.csv', 'traditional_w_coordinate_18000-18100.csv', 'traditional_w_coordinate_11500-11600.csv', 'traditional_w_coordinate_1600-1700.csv', 'traditional_w_coordinate_7100-7200.csv', 'traditional_w_coordinate_3000-3100.csv', 'traditio

,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
0,28876,319,화양제일골목시장(화양제일시장),화양식품,서울특별시 광진구 아차산로29길 59(화양동)1층,02-465-6374,두부,37.543482,127.069282,POINT (127.06928 37.54348)
1,28877,318,화양제일골목시장(화양제일시장),레드크라운,서울특별시 광진구 아차산로29길 56(화양동)1층,02-3409-5545,도소매(케이스),37.543187,127.069398,POINT (127.06940 37.54319)
2,28878,317,화양제일골목시장(화양제일시장),해안수산,서울 광진구 동일로24길 70(화양동),NaN,농축수산품,37.542833,127.068790,POINT (127.06879 37.54283)
3,28879,316,화양제일골목시장(화양제일시장),신토불이청과,서울특별시 광진구 동일로24길 64(화양동)1층,02-463-6740,농산물,37.542940,127.068531,POINT (127.06853 37.54294)
4,28880,315,화양제일골목시장(화양제일시장),제일상회,서울특별시 광진구 아차산로29길 59(화양동) 1층,02-465-3067,농산물,37.543482,127.069282,POINT (127.06928 37.54348)


In [39]:
katec = "+proj=tmerc +lat_0=38 +lon_0=127.0028902777778 +k=1 +x_0=200000 +y_0=500000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43"
gdf_by_naver = csvToDataFrame("./data/geocodeByNaver/",katec)
gdf_by_naver['lat'] = gdf_by_naver.to_crs(4326).geometry.y
gdf_by_naver

['filled_by_naver.csv']


,Unnamed: 0.1,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
0,11,28887,308,황학동 주방가구거리 상점가,씨유(CU) 신당메트하임점,"서울 중구 마장로 1161층 101호(황학동, 신당메트하임오피스텔)",NaN,편의점,inf,1.270227e+09,POINT (1270227031.000 375678794.000)
1,16,28892,303,황학동 주방가구거리 상점가,건영주방,서울 중구 마장로 1011층 연풍주차장 (황학동),NaN,주방용식기,inf,1.270199e+09,POINT (1270198965.000 375684684.000)
2,17,28893,302,황학동 주방가구거리 상점가,모아종합주방,서울 중구 마장로 1081층(황학동),NaN,주방용식기,inf,1.270216e+09,POINT (1270216332.000 375680564.000)
3,19,28895,300,황학동 주방가구거리 상점가,세한종합주방,서울 중구 마장로 791층 (황학동),NaN,주방용식기,inf,1.270185e+09,POINT (1270184730.000 375684880.000)
4,21,28898,298,황학동 주방가구거리 상점가,프렌드주방시스템,서울 중구 마장로 991층 (황학동),NaN,주방용식기,NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...
4225,27744,17468,11183,선유도역골목형상점가,앙앤온,서울 영등포구 양평로19길 36층 (양평동4가),NaN,기타음료식품,inf,1.268951e+09,POINT (1268950670.000 375368344.000)
4226,27745,17469,11182,선유도역골목형상점가,원띵 비헬스,"서울 영등포구 선유로49길 236층 603호 (양평동4가, 선유도역2차아이에스비즈타워)",NaN,기타건강식,NaN,NaN,POINT EMPTY
4227,27749,17474,11178,성내전통시장,웰스마켓,"서울 강동구 천호대로162길 7310층 1002호 (성내동, 원다빌2)",NaN,홍삼제품,NaN,NaN,POINT EMPTY
4228,27758,17483,11169,성내전통시장,강동 정육식당,서울 강동구 천호대로162길 671층 (성내동),NaN,일반한식,inf,1.271278e+09,POINT (1271277966.000 375338736.000)


In [40]:
tm128_to_wgs84(1270198965,375684684)

/var/folders/g4/w09fq4gd4qs138521vh0xk2w0000gn/T/ipykernel_2862/1588324624.py:2: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  return transform( Proj(**TM128), Proj(**WGS84), x, y )


(inf, inf)

# Load Csv as pandas then transform into gdf

In [41]:
df_by_naver = pd.read_csv("./data/geocodeByNaver/filled_by_naver.csv")
df_by_naver = df_by_naver.drop(["geometry"],axis=1)
df_by_naver[["lat","lng"]] = df_by_naver[["lat","lng"]].apply(lambda x: x/10**7)
df_by_naver

,Unnamed: 0.1,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng
0,11,28887,308,황학동 주방가구거리 상점가,씨유(CU) 신당메트하임점,"서울 중구 마장로 1161층 101호(황학동, 신당메트하임오피스텔)",NaN,편의점,37.567879,127.022703
1,16,28892,303,황학동 주방가구거리 상점가,건영주방,서울 중구 마장로 1011층 연풍주차장 (황학동),NaN,주방용식기,37.568468,127.019897
2,17,28893,302,황학동 주방가구거리 상점가,모아종합주방,서울 중구 마장로 1081층(황학동),NaN,주방용식기,37.568056,127.021633
3,19,28895,300,황학동 주방가구거리 상점가,세한종합주방,서울 중구 마장로 791층 (황학동),NaN,주방용식기,37.568488,127.018473
4,21,28898,298,황학동 주방가구거리 상점가,프렌드주방시스템,서울 중구 마장로 991층 (황학동),NaN,주방용식기,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4225,27744,17468,11183,선유도역골목형상점가,앙앤온,서울 영등포구 양평로19길 36층 (양평동4가),NaN,기타음료식품,37.536834,126.895067
4226,27745,17469,11182,선유도역골목형상점가,원띵 비헬스,"서울 영등포구 선유로49길 236층 603호 (양평동4가, 선유도역2차아이에스비즈타워)",NaN,기타건강식,NaN,NaN
4227,27749,17474,11178,성내전통시장,웰스마켓,"서울 강동구 천호대로162길 7310층 1002호 (성내동, 원다빌2)",NaN,홍삼제품,NaN,NaN
4228,27758,17483,11169,성내전통시장,강동 정육식당,서울 강동구 천호대로162길 671층 (성내동),NaN,일반한식,37.533874,127.127797


# turn to gdf

In [42]:
gdf_by_naver = gpd.GeoDataFrame(
    df_by_naver, geometry=gpd.points_from_xy(df_by_naver.lng,df_by_naver.lat),crs="EPSG:4326"
)
gdf_by_naver

,Unnamed: 0.1,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry
0,11,28887,308,황학동 주방가구거리 상점가,씨유(CU) 신당메트하임점,"서울 중구 마장로 1161층 101호(황학동, 신당메트하임오피스텔)",NaN,편의점,37.567879,127.022703,POINT (127.02270 37.56788)
1,16,28892,303,황학동 주방가구거리 상점가,건영주방,서울 중구 마장로 1011층 연풍주차장 (황학동),NaN,주방용식기,37.568468,127.019897,POINT (127.01990 37.56847)
2,17,28893,302,황학동 주방가구거리 상점가,모아종합주방,서울 중구 마장로 1081층(황학동),NaN,주방용식기,37.568056,127.021633,POINT (127.02163 37.56806)
3,19,28895,300,황학동 주방가구거리 상점가,세한종합주방,서울 중구 마장로 791층 (황학동),NaN,주방용식기,37.568488,127.018473,POINT (127.01847 37.56849)
4,21,28898,298,황학동 주방가구거리 상점가,프렌드주방시스템,서울 중구 마장로 991층 (황학동),NaN,주방용식기,NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...
4225,27744,17468,11183,선유도역골목형상점가,앙앤온,서울 영등포구 양평로19길 36층 (양평동4가),NaN,기타음료식품,37.536834,126.895067,POINT (126.89507 37.53683)
4226,27745,17469,11182,선유도역골목형상점가,원띵 비헬스,"서울 영등포구 선유로49길 236층 603호 (양평동4가, 선유도역2차아이에스비즈타워)",NaN,기타건강식,NaN,NaN,POINT EMPTY
4227,27749,17474,11178,성내전통시장,웰스마켓,"서울 강동구 천호대로162길 7310층 1002호 (성내동, 원다빌2)",NaN,홍삼제품,NaN,NaN,POINT EMPTY
4228,27758,17483,11169,성내전통시장,강동 정육식당,서울 강동구 천호대로162길 671층 (성내동),NaN,일반한식,37.533874,127.127797,POINT (127.12780 37.53387)


# now concat gdfs

In [43]:
gdfs = [gdf_by_sgis,gdf_by_naver]
all_gdf = pd.concat(gdfs)
all_gdf

,Unnamed: 0,번호,시장명,상호명,주소,연락처,취급품목(업종),lat,lng,geometry,Unnamed: 0.1
0,28876,319,화양제일골목시장(화양제일시장),화양식품,서울특별시 광진구 아차산로29길 59(화양동)1층,02-465-6374,두부,37.543482,127.069282,POINT (127.06928 37.54348),NaN
1,28877,318,화양제일골목시장(화양제일시장),레드크라운,서울특별시 광진구 아차산로29길 56(화양동)1층,02-3409-5545,도소매(케이스),37.543187,127.069398,POINT (127.06940 37.54319),NaN
2,28878,317,화양제일골목시장(화양제일시장),해안수산,서울 광진구 동일로24길 70(화양동),NaN,농축수산품,37.542833,127.068790,POINT (127.06879 37.54283),NaN
3,28879,316,화양제일골목시장(화양제일시장),신토불이청과,서울특별시 광진구 동일로24길 64(화양동)1층,02-463-6740,농산물,37.542940,127.068531,POINT (127.06853 37.54294),NaN
4,28880,315,화양제일골목시장(화양제일시장),제일상회,서울특별시 광진구 아차산로29길 59(화양동) 1층,02-465-3067,농산물,37.543482,127.069282,POINT (127.06928 37.54348),NaN
...,...,...,...,...,...,...,...,...,...,...,...
4225,17468,11183,선유도역골목형상점가,앙앤온,서울 영등포구 양평로19길 36층 (양평동4가),NaN,기타음료식품,37.536834,126.895067,POINT (126.89507 37.53683),27744.0
4226,17469,11182,선유도역골목형상점가,원띵 비헬스,"서울 영등포구 선유로49길 236층 603호 (양평동4가, 선유도역2차아이에스비즈타워)",NaN,기타건강식,NaN,NaN,POINT EMPTY,27745.0
4227,17474,11178,성내전통시장,웰스마켓,"서울 강동구 천호대로162길 7310층 1002호 (성내동, 원다빌2)",NaN,홍삼제품,NaN,NaN,POINT EMPTY,27749.0
4228,17483,11169,성내전통시장,강동 정육식당,서울 강동구 천호대로162길 671층 (성내동),NaN,일반한식,37.533874,127.127797,POINT (127.12780 37.53387),27758.0


# create markered maps

In [44]:
gdf_for_marker = all_gdf[all_gdf["lat"].notna()]
locs = gdf_for_marker[["lat","lng"]]
lats = np.array(locs["lat"])
lngs = np.array(locs["lng"])
locations = list(zip(lats,lngs))
popups = gdf_for_marker[["시장명","상호명","취급품목(업종)","주소"]]

m = folium.Map([37.533297,127.127313],
                tiles="Cartodb Positron",
               zoom_start=12)

marker_cluster = MarkerCluster(
    name="Traditional Markets",
    overlay=True,
    control=True
)

popups_lst = popups.values.tolist()
for ((mk_name,store_name,category,adrs),(lat,lng)) in zip(popups_lst,locations):
    cur_loc = lat,lng
    marker = folium.Marker(location=cur_loc,name=store_name+"/"+str(category),cat=category)
    popup = f'<h3>{store_name},{category}</h3><b>시장</b>:{mk_name}<br><b>주소</b>:{adrs}'
    folium.Popup(popup,max_width=300).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

name_search = Search(
    layer=marker_cluster,
    search_label="name",
    placeholder="이름을 입력하세요",
    collapsed=False,
).add_to(m)



folium.LayerControl().add_to(m)
m.save("./market_clustered_by_all.html")